In [3]:
import requests
from pydantic import BaseModel
from typing import Type, TypeVar

T = TypeVar("T", bound=BaseModel)


def ask_ollama_with_schema(
    model_cls: Type[T], user_prompt: str, model_name: str = "llama3.1"
) -> T:
    """
    Sends a request to Ollama's chat API with a JSON schema derived from a Pydantic model.

    :param model_cls: Pydantic model class defining the expected output format.
    :param user_prompt: The prompt to send to the model.
    :param model_name: Ollama model to use.
    :return: An instance of model_cls containing the model's structured output.
    """
    url = "http://localhost:11434/api/chat"
    headers = {"Content-Type": "application/json"}

    # Get JSON Schema from Pydantic
    schema = model_cls.model_json_schema()

    payload = {
        "model": model_name,
        "messages": [{"role": "user", "content": user_prompt}],
        "stream": False,
        "format": schema,  # Pass the schema directly
    }

    resp = requests.post(url, headers=headers, json=payload)
    resp.raise_for_status()
    data = resp.json()

    # Ollama usually returns { "message": { "content": "{...}" }, ... }
    try:
        content = data["message"]["content"]
        if isinstance(content, str):
            import json

            content = json.loads(content)
    except (KeyError, ValueError) as e:
        raise RuntimeError(f"Unexpected response format: {data}") from e

    return model_cls(**content)


# Example usage:
class CountryInfo(BaseModel):
    name: str
    capital: str
    languages: list[str]


if __name__ == "__main__":
    country = ask_ollama_with_schema(CountryInfo, "Tell me about Canada.")
    print(country)

HTTPError: 404 Client Error: Not Found for url: http://localhost:11434/api/chat